# Create Heat Map for significant DNA Replication genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as pu

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [ ]:
df = pd.read_csv('sig_pval_all_proteins.csv')

prot_list = list(df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [12]:
prot_enr.res2d.head(15)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,84/134,1.760059e-41,5.156972e-39,0,0,4.851901,455.305549,RBM25;EIF4A3;DDX42;HNRNPU;PRPF19;USP39;EFTUD2;...,KEGG_2016
1,RNA transport Homo sapiens hsa03013,76/172,3.577573e-24,5.241145e-22,0,0,3.419973,184.635286,CYFIP2;NUP107;NUP188;RPP30;EIF4A3;PHAX;PNN;XPO...,KEGG_2016
2,DNA replication Homo sapiens hsa03030,22/36,1.580044e-11,1.543176e-09,0,0,4.729962,117.638810,RFC5;FEN1;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;PRIM1;...,KEGG_2016
3,mRNA surveillance pathway Homo sapiens hsa03015,34/91,3.038588e-09,2.225765e-07,0,0,2.891845,56.714497,SMG1;RNMT;EIF4A3;MSI1;PPP2R2A;MSI2;SMG7;SMG6;P...,KEGG_2016
4,Mismatch repair Homo sapiens hsa03430,15/23,7.918079e-09,4.639994e-07,0,0,5.047786,94.161986,RFC5;RFC3;PCNA;RFC4;RFC1;RFC2;RPA1;MLH1;POLD3;...,KEGG_2016
5,Ribosome biogenesis in eukaryotes Homo sapiens...,32/89,2.582651e-08,1.261194e-06,0,0,2.782899,48.622436,NVL;WDR3;RPP30;HEATR1;NAT10;NMD3;PWP2;WDR43;NO...,KEGG_2016
6,Epstein-Barr virus infection Homo sapiens hsa0...,53/202,2.467229e-07,1.032712e-05,0,0,2.030776,30.898254,ATF2;CDKN1B;AKAP8L;PIK3CD;ITGAL;PIK3CG;ICAM1;P...,KEGG_2016
7,Fc gamma R-mediated phagocytosis Homo sapiens ...,31/93,3.051032e-07,1.117441e-05,0,0,2.579979,38.706439,WAS;PIK3CD;ASAP1;PIK3R1;PLD1;PIK3CG;PAK1;SCIN;...,KEGG_2016
8,Pyrimidine metabolism Homo sapiens hsa00240,32/105,1.880792e-06,6.123022e-05,0,0,2.358838,31.098494,DTYMK;PRIM1;NUDT2;NT5C2;POLD3;POLR2B;POLD1;POL...,KEGG_2016
9,Osteoclast differentiation Homo sapiens hsa04380,37/132,2.903409e-06,8.506989e-05,0,0,2.169528,27.660670,CSF1R;PIK3CD;TREM2;PIK3R1;PIK3CG;RELB;PPP3CB;A...,KEGG_2016


# Step 2: Get the list of significant genes in the DNA Replication pathway

In [3]:
dna_rep = prot_enr.res2d.Genes[2]
genes = dna_rep.split(';')
print('total genes:',len(genes))

total genes: 22


# Step 3: Create HeatMap

Create a long df formated for the plotCircleHeatMap function (read in heat_map_df.csv). Then slice out about 15 genes from the significant genes in the DNA Replication pathway. 15 genes show up well when the plotCircleHeatMap function is called. Change the variable representing a different list of genes to visualize all the genes in the pathway.  

In [4]:
hm_df = pd.read_csv('heat_map_df.csv') # long df plottable with HeatMap function

In [5]:
genes_1 = genes[0:16]
genes_2 = genes[16:32]

In [6]:
bool_df = hm_df.Proteomics.isin(genes_1)
plot_df = hm_df[bool_df]
plot_df.head()

,Proteomics,P_Value,Medians,Cancer,size
131,PCNA,0.000019,-0.631486,Gbm,1.087410
133,POLD2,0.000020,-0.405249,Gbm,1.081765
178,POLD3,0.000036,-0.483613,Gbm,1.022357
179,RFC5,0.000037,-0.481635,Gbm,1.021302
220,MCM7,0.000053,-1.036455,Gbm,0.984818


In [7]:
pu.plotCircleHeatMap(plot_df, circle_var = 'size', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer')

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = np.where(df[circle_var]<0, np.abs(df[circle_var]), df[circle_var])*50


Loading BokehJS ...

# Step 4: Create a HeatMap with both pos and neg differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [8]:
pos_neg_df = pd.read_csv('pos_neg_df.csv')

In [9]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2',
       'RFC4', 'SSBP1', 'POLA2'], dtype=object)

In [10]:
pu.plotCircleHeatMap(genes_pn, circle_var = 'size', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer')

Loading BokehJS ...